# 🔌 Simple MCP Example

A demonstration of connecting an AI agent to MCP (Model Context Protocol) servers for web search and file management.

---

- 📋 Overview
    - 🌍 **Task:** Web search and file management using MCP servers
    - 🧠 **Model:** OpenAI GPT-4o-mini
    - 🎯 **Process:** Setup MCP servers → Connect to agent → Execute search & file tasks
    - 📌 **Output Format:** Agent responses and saved files in sandbox folder
    - 🔧 **Tools:** Agents SDK, MCP servers (Serper, Filesystem), uvx, npx
    - 🧑‍💻 **Skill Level:** Intermediate

- ⚠️ **Windows Users:** This notebook requires WSL (Windows Subsystem for Linux) to run properly. See [`13_guide_wsl.md`](https://github.com/lisekarimi/agentverse/blob/main/6_mcp/13_guide_wsl.md) for setup instructions.


- 🛠️ Requirements
    - ⚙️ **Hardware:** ✅ CPU is sufficient
    - 🔑 **OpenAI API Key**
    - 🔍 **Serper API Key** (for Google search)
    - 📦 **uvx** (Python package runner - comes with uv)
    - 📦 **npx** (Node.js package runner - comes with Node.js)
    - The `sandbox/` folder will be created automatically for file operations

---
📢 Discover more Agentic AI notebooks on my [GitHub repository](https://github.com/lisekarimi/agentverse) and explore additional AI projects on my [portfolio](https://lisekarimi.com).


## Imports and settings

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

load_dotenv(override=True)

MODEL = "gpt-4o-mini"

## MCP server CONFIGURATIONS

In [2]:
# Serper MCP Server setup
serper_params = {
    "command": "uvx",  # This downloads the server to your machine
    "args": ["serper-mcp-server"], # Package name to download and run from PyPI
    "env": {
        "SERPER_API_KEY": os.getenv("SERPER_API_KEY")
    }
}

When you run this ☝️:
- uvx downloads serper-mcp-server from PyPI
- Installs it locally on your machine
- Runs it as a local process
- Your code communicates with it via stdio (same machine)
- The local MCP server then makes API calls to remote Serper

In [ ]:
async with MCPServerStdio(params=serper_params, client_session_timeout_seconds=60) as serper_server:
    serper_tools = await serper_server.session.list_tools()

serper_tools.tools

In [3]:
# File system MCP Server
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))  # Create absolute path to sandbox folder

# Create sandbox directory if it doesn't exist
os.makedirs(sandbox_path, exist_ok=True)

files_params = {
    "command": "npx",  # Node package runner
    "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]  # Download filesystem MCP server and restrict it to sandbox folder
}

In [ ]:
async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as files_server:
    files_tools = await files_server.session.list_tools()

files_tools.tools

This MCP server gives the AI agent the ability to read, write, create, and manage files, but only within the sandbox folder for security.

## Create and run agent

In [4]:
instructions = """
You search Google and save files to the sandbox folder.
Use write_file tool when saving content to files.
Use Pushover only when user requests notifications.
"""

In [5]:
async def create_and_run_agent(task: str):

    # Create MCP clients (subprocess)
    async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_files:
        async with MCPServerStdio(params=serper_params, client_session_timeout_seconds=60) as mcp_search:

                # Create agent with MCP clients
                agent = Agent(
                    name="investigator",
                    instructions=instructions,
                    model=MODEL,
                    mcp_servers=[mcp_files, mcp_search]
                )

                # Run the agent with tracing
                with trace("investigate"):
                    result = await Runner.run(agent, task)
                    return result

In [6]:
result = await create_and_run_agent("Search for the top 5 highest paying data scientist jobs currently available in the New York area, then save the list with job titles and companies to data_scientist_jobs.md")
print(result.final_output)

The list of the top 5 highest paying data scientist jobs in New York has been successfully saved to `sandbox/data_scientist_jobs.md`. Here are the details included in the file:

1. **Data Scientist** at **Airbnb** - $246K - $390K per year
2. **Data Scientist** at **StreetEasy** - $245K - $391K per year
3. **Data Scientist** at **Coinbase** - $245K - $372K per year
4. **Data Scientist** at **Stripe** - $236K - $363K per year
5. **Data Scientist** at **Block** - $226K - $351K per year

If you need any further assistance, feel free to ask!
